# Real Choropleths

In the second part of the workshop, we will cover how to process data to generate the choropleths, including file formats, spatial reference systems, table joins and color stops.

We will use the following datasets, courtesy of [NYC Open Data](https://data.cityofnewyork.us).

- [NYC Zip Code Boundaries](https://data.cityofnewyork.us/Business/Zip-Code-Boundaries/i8iw-xf4u)
- [DSNY Graffiti Trafficking](https://data.cityofnewyork.us/City-Government/DSNY-Graffiti-Tracking/gpwd-npar)

For more 2D spatial data, please see [NYC DoITT 2D Data](https://www1.nyc.gov/site/doitt/residents/gis-2d-data.page).

There are now several packages you can use to process spatial vector and raster data:

- [fiona](https://fiona.readthedocs.io)
- [geopandas](https://geopandas.readthedocs.io)
- [geotable](https://github.com/invisibleroads/geotable)
- [rasterio](https://rasterio.readthedocs.io)

TODO: Work on variations of this notebook with different packages

In [ ]:
pip install geotable --user

In [ ]:
from os import makedirs

datasets_folder = 'datasets'
try:
    makedirs(datasets_folder)
except OSError:
    pass

In [ ]:
rm $datasets_folder/*

## File Formats

Standard file formats for geospatial data include geojson, shapefile, kml and kmz.

### Exercise: Determine File Format

1. Visit [NYC Police Precincts](https://data.cityofnewyork.us/Public-Safety/Police-Precincts/78dh-3ptz).
2. Click the Export tab and download each of the geospatial file formats.
3. Note the file extension and file size for each file format. 
4. Visit [NYC Zip Code Boundaries](https://data.cityofnewyork.us/Business/Zip-Code-Boundaries/i8iw-xf4u).
5. Download the file and examine its contents to determine its file format.

In [ ]:
# Load NYC zip codes via URL using geotable
import geotable
url = 'https://data.cityofnewyork.us/download/i8iw-xf4u/application%2Fzip'
t = raw_nyc_zip_code_table = geotable.load(url)
len(t)

In [ ]:
# Show the first two rows
t[:2]

In [ ]:
# Render geometries
t.draw()

In [ ]:
# Render each geometry with a random color
from geotable import ColorfulGeometryCollection
ColorfulGeometryCollection(t.geometries)

In [ ]:
from os.path import join

# Save as a geojson
nyc_zip_codes_geojson_path = t.save_geojson(join(
    datasets_folder, 'nyc-zip-codes.geojson'))

# Save as a shapefile using the longitude latitude coordinate system
t.save_shp(join(
    datasets_folder, 'nyc-zip-codes.shp.zip',
), target_proj4=geotable.LONGITUDE_LATITUDE_PROJ4)

In [ ]:
ll $datasets_folder -h

In [ ]:
raw_nyc_zip_code_table.iloc[0]

In [ ]:
# Extract specific columns
raw_nyc_zip_code_table = raw_nyc_zip_code_table[[
    'ZIPCODE', 'POPULATION', 'AREA', 'geometry_object', 'geometry_proj4']]
raw_nyc_zip_code_table.iloc[0]

In [ ]:
# Get geojson so that we can plot choropleth
import json
d = json.load(open(nyc_zip_codes_geojson_path, 'rt'))

In [ ]:
# Compute stops
raw_nyc_zip_code_table['AREA'].describe()

In [ ]:
area_stops = raw_nyc_zip_code_table['AREA'].describe()[3:].tolist()
area_stops

In [ ]:
from mapboxgl.utils import create_color_stops, create_numeric_stops
from mapboxgl.viz import ChoroplethViz
from os import getenv

MAPBOX_TOKEN = getenv('MAPBOX_TOKEN', 'YOUR-MAPBOX-TOKEN')
MAPBOX_TOKEN = 'pk.eyJ1IjoiY3Jvc3Njb21wdXRlIiwiYSI6ImNrczRwNTVuZTEzeG8ydXBrbjY4Ym1iaTcifQ.UidJO47zokdyOa2huvaWBA'

v = ChoroplethViz(
    d,
    style='mapbox://styles/mapbox/dark-v10',    
    access_token=MAPBOX_TOKEN,
    color_property='AREA',
    color_stops=create_color_stops(area_stops, colors='Greens'),
    color_function_type='interpolate',
    line_stroke='--',
    line_color='Yellow',
    line_width=1,
    line_opacity=0.9,
    opacity=0.8,
    center=center_coordinates,
    zoom=9,
    below_layer='waterway-label',
    legend_layout='horizontal',
    legend_key_shape='bar',
    legend_key_borders_on=False)
v.show()

### Spotlight: Reduce GeoJSON Size

For huge datasets, there are a few strategies for reducing GeoJSON file size. Smaller GeoJSON files can speed your analysis and visualizations.

1. Simplify geometries (see https://shapely.readthedocs.io/en/stable/manual.html#object.simplify).

TODO: Add simplification notes

2. Extract specific columns.

```python
raw_nyc_zip_code_table = raw_nyc_zip_code_table[[
    'ZIPCODE', 'POPULATION', 'AREA', 'geometry_object', 'geometry_proj4']]
```

## Spatial Reference Systems

A spatial reference system assigns coordinates to each location.

In [ ]:
# Load geojson as a table
t = nyc_zip_code_table = geotable.load(join(datasets_folder, 'nyc-zip-codes.geojson'))
t[:2]

### Exercise: Explore Spatial Reference Systems

1. Open https://spatialreference.org.
2. Find the search box in the upper right corner.
3. Enter New York and press ENTER.
4. Click on a result such as https://spatialreference.org/ref/epsg/2263/.
5. Click Proj4.

### Different Systems, Different Coordinates

- Different spatial reference systems assign different coordinates to the same location.
- A region such as New York has many options when choosing a spatial reference system.
- Different spatial references systems serve different purposes.
- Regional spatial references have greater accuracy and speed for distance calculations when all locations are within the prescribed region because we are assuming the surface is flat.
- Global spatial references such as longitude latitude have greater accuracy for transregional distance calculations, but may require using a special function to calculate distances because we are assuming the surface is curved.
- Some spatial references such as spherical mercator distort distances and areas because they are optimized for viewing the earth as a flat map.

In [ ]:
raw_nyc_zip_code_table[['ZIPCODE', 'geometry_object', 'geometry_proj4']][:2]

In [ ]:
nyc_zip_code_table[['ZIPCODE', 'geometry_object', 'geometry_proj4']][:2]

### Specifying Your Spatial Reference System Using PROJ

There are many ways to specify which spatial reference system you are using to assign coordinates to your locations. Today, we will use a PROJ string to specify the spatial reference.

In [ ]:
# See one of the many spatial references for New York
raw_nyc_zip_code_table['geometry_proj4'][0]

In [ ]:
# See the longitude latitude spatial reference
nyc_zip_code_table['geometry_proj4'][0]

In [ ]:
geotable.projections.LONGITUDE_LATITUDE_PROJ4

In [ ]:
geotable.projections.SPHERICAL_MERCATOR_PROJ4

### Spotlight: Calculate Distances in Different Spherical Projections

TODO: Mention different distance functions in geopy as well as google api

### Spotlight: Compute Center Coordinates

In [ ]:
# See center coordinates in original spatial reference
from shapely.geometry import GeometryCollection
GeometryCollection(t.geometries).centroid.coords[0]

In [ ]:
import json
d = json.load(open(nyc_zip_codes_geojson_path, 'rt'))
d['features'][0]['properties']

In [ ]:
# Load shapely geometries from geojson
from shapely.geometry import shape
shape(d['features'][0]['geometry'])

In [ ]:
from shapely.geometry import GeometryCollection
collection = GeometryCollection([shape(_['geometry']) for _ in d['features']])
center_coordinates = collection.centroid.coords[0]
center_coordinates

## Table Joins

In [ ]:
import pandas as pd

In [ ]:
url = 'https://data.cityofnewyork.us/api/views/gpwd-npar/rows.csv'
nyc_graffiti_table = pd.read_csv(url, dtype=str)
len(nyc_graffiti_table)

In [ ]:
nyc_graffiti_table.iloc[0]

In [ ]:
nyc_graffiti_counts_table = nyc_graffiti_table[[
    'STATUS', 'ZIP_CODE',
]].groupby('ZIP_CODE').count()
nyc_graffiti_counts_table = nyc_graffiti_counts_table.rename({
    'STATUS': 'GRAFFITI_COUNT'}, axis=1)
nyc_graffiti_counts_table

In [ ]:
joined_table = pd.merge(
    raw_nyc_zip_code_table,
    nyc_graffiti_counts_table,
    left_on='ZIPCODE',
    right_on='ZIP_CODE')

In [ ]:
joined_table['GRAFFITI_SCORE'] = joined_table['GRAFFITI_COUNT'] / joined_table['AREA'] * 1000000

In [ ]:
dict(joined_table.iloc[0])

## Color Stops

In [ ]:

t['AREA'].describe()

In [ ]:
t['POPULATION'].describe()

In [ ]:
t['AREA'].describe()[3:]

In [ ]:
t['AREA'].describe()[3:].tolist()

## Full Example

In [ ]:
# Load shapefile
import geotable
url = 'https://data.cityofnewyork.us/download/i8iw-xf4u/application%2Fzip'
raw_nyc_zip_code_table = geotable.load(url)
raw_nyc_zip_code_table = raw_nyc_zip_code_table[[
    'ZIPCODE', 'POPULATION', 'AREA', 'geometry_object', 'geometry_proj4']]
raw_nyc_zip_code_table

In [ ]:
# Add data
raw_nyc_zip_code_table['DENSITY'] = raw_nyc_zip_code_table[
    'POPULATION'] / raw_nyc_zip_code_table['AREA']

In [ ]:
# Join data
import pandas as pd
url = 'https://data.cityofnewyork.us/api/views/gpwd-npar/rows.csv'
nyc_graffiti_table = pd.read_csv(url, dtype='str')
nyc_graffiti_counts_table = nyc_graffiti_table[[
    'STATUS', 'ZIP_CODE',
]].groupby('ZIP_CODE').count()
nyc_graffiti_counts_table = nyc_graffiti_counts_table.rename({
    'STATUS': 'GRAFFITI_COUNT'}, axis=1)
joined_table = pd.merge(
    raw_nyc_zip_code_table,
    nyc_graffiti_counts_table,
    left_on='ZIPCODE',
    right_on='ZIP_CODE')
joined_table['GRAFFITI_SCORE'] = joined_table['GRAFFITI_COUNT'] / joined_table['AREA'] * 1000000
joined_table

In [ ]:
# Get geojson
import json
joined_geojson_path = joined_table[[
    'ZIPCODE',
    'DENSITY',
    'GRAFFITI_SCORE',
    'geometry_object',
    'geometry_proj4',
]].save_geojson(join(datasets_folder, 'd.geojson'))
d = json.load(open(joined_geojson_path, 'rt'))
len(d['features'])

In [ ]:
# Compute stops
graffiti_score = joined_table['GRAFFITI_SCORE']
graffiti_score_stops = graffiti_score.describe()[3:].tolist()

density = joined_table['DENSITY']
density_stops = density.describe()[3:].tolist()

In [ ]:
# Compute center coordinates
from shapely.geometry import GeometryCollection, shape
collection = GeometryCollection([shape(_['geometry']) for _ in d['features']])
center_coordinates = collection.centroid.coords[0]
center_coordinates

In [ ]:
# Render choropleth
from mapboxgl.utils import create_color_stops, create_numeric_stops
from mapboxgl.viz import ChoroplethViz
from os import getenv

MAPBOX_TOKEN = getenv('MAPBOX_TOKEN', 'YOUR-MAPBOX-TOKEN')

v = ChoroplethViz(
    d,
    style='mapbox://styles/mapbox/dark-v10',    
    access_token=MAPBOX_TOKEN,
    color_property='GRAFFITI_SCORE',
    color_stops=create_color_stops(graffiti_score_stops, colors='Reds'),
    # color_property='DENSITY',
    # color_stops=create_color_stops(density_stops, colors='Blues'),
    color_function_type='interpolate',
    line_stroke='--',
    line_color='black',
    line_width=1,
    line_opacity=0.9,
    opacity=0.8,
    center=center_coordinates,
    zoom=9,
    below_layer='waterway-label',
    legend_layout='horizontal',
    legend_key_shape='bar',
    legend_key_borders_on=False)
# v.bearing = 30
# v.pitch = 30
# v.height_property = 'DENSITY'
# v.height_stops = create_numeric_stops(density_stops, 0, 5000)
# v.height_function_type = 'interpolate'
v.show()